In [18]:
#import tensorlow library
import tensorflow as tf

In [19]:
hello = tf.constant('Hello, Tensorflow!')
print(hello)

Tensor("Const_3:0", shape=(), dtype=string)


##  3-1 텐서 
 : 텐서플로에서 가장 기본적인 자료형으로 Rank와 Shape의 개념을 가짐
 
 * 랭크(Rank)
  * 차원의 수를 나타냄 
  * 0이면 스칼라, 1이면 벡터, 2는 행령 3이상은 n-Tensor (n차원 텐서)
  
 * 셰이프(Shape)
  * 각 차원의 요소 개수 ( 텐서의 구조를 설명 ) 

In [20]:
a= tf.constant(10)
b= tf.constant(40)
c=tf.add(a,b)
print(c)

Tensor("Add_2:0", shape=(), dtype=int32)


### 프로그램의 구조

#### 1. 그래프 생성
: 텐서와 텐서의 연사들을 먼저 정의하여 그래프를 만든다

#### 2. 그래프 실행 
: 원하는 시점에 실제 연산을 수행 => Session 안에서 이루어짐.

In [21]:
sess = tf.Session()

print(sess.run(hello))
print(sess.run([a,b,c]))

sess.close()

b'Hello, Tensorflow!'
[10, 40, 50]


## 3-2 플레이스홀더와 변수

#### 1. 플레이스홀더 
: 그래프에 사용할 입력값을 나중에 받기 위해 사용하는 parameter(매개변수)

#### 2. 변수
: 그래프를 최적화 하는 용도로 학습 함수들이 학습한 결과를 갱신하기 위함 
 * 이 변수 값들이 신경망의 성능을 좌우함

In [22]:
# (?,3) 모양의 float32 자료형을 가진 텐서 생성 
X = tf.placeholder(tf.float32,[None,3])
print(X)

x_data = [[1,2,3],[4,5,6]]

# [3,2] 행렬 형태의 탠서
## tf.random_normal 함수로 정규분포의 무작위 값으로 초기화
W= tf.Variable(tf.random_normal([3,2]))

# [2,1] 행렬 형태의 텐서 
b= tf.Variable(tf.random_normal([2,1]))

#행렬곱 연산 (tf.matmul)
expr = tf.matmul(X,W)+b

Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)


### 행렬곱 

* 행렬곱 A X B에 대하여, 행렬 A의 열 수와 행렬 B의 행 수는 같아야 한다.

* 행렬곱 A X B를 계산한 행렬 AB의 크기는 A의 행 개수와 B의 열 개수가 된다.

In [23]:
sess = tf.Session()
#앞에서 정의한 변수들을 초기화하는 함수(global_variables_initiallizer())
sess.run(tf.global_variables_initializer())

print("====x_data====")
print(x_data)
print("====W====")
print(sess.run(W))
print("====b====")
print(sess.run(b))
print("====expr====")
#feed_dict : 그래프를 실행할 때 사용할 입력값 지정(변수임)
print(sess.run(expr, feed_dict={X: x_data}))

sess.close()

====x_data====
[[1, 2, 3], [4, 5, 6]]
====W====
[[ 1.4707674  -1.7815707 ]
 [ 0.17070957  1.5098804 ]
 [-1.1469615  -1.4239373 ]]
====b====
[[-0.36351264]
 [ 0.71006644]]
====expr====
[[-1.9922105 -3.3971343]
 [ 0.5649148 -7.410438 ]]


## 3-3 선형 회귀 모델 구현하기 

### 1. 선형 회귀란( linear regression)
: 주어진 x와 y의 값을 가지고 서로 간의 **관계(상관 관계)를 파악**하는 것 \
=> 즉, 어떤 입력값에 대한 출력을 예측하는 것이다. 


In [25]:
x_data =[1,2,3]
y_data =[1,2,3]

W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

X = tf.placeholder(tf.float32, name ="X")
Y = tf.placeholder(tf.float32, name ="Y")

# W:가중치 , b: 편향 
hypothesis = W * X + b





### 2. 손실함수 (loss function)
: 한 쌍의 (x,y)의 데이터에 대한 손실값을 계산하는 함수 

* **손실값** \
실제값과 모델로 예측한 값이 얼마나 차이가 나는가 나타내는 값 \
 \: 손실값이 작을수록 그 모델이 X와 Y의 관계 잘 설명하고 있다는 의미 
    + 손실값으로는 예측값과 실제값의 거리를 가장 많이 사용함


* **비용** \
:손실을 전체 데이터에 대해 구한 경우 
    + 모든 데이터에 대한 손실값의 평균을 구함


* **학습**
: 변수들의 값을 다양하게 넣어 계산해보면서 손실값을 최소화하는 W와 b의 값을 구하는 것 


In [26]:
cost = tf.reduce_mean(tf.square(hypothesis - Y))

### 3. 최적화함수 - 경사하강법(gradient descent)
: 손실값을 최소화하는 연산 그래프 생성 

* **최적화 함수란?** \
: 가중치와 편향 값을 변경해가면서 손실값을 최소화하는 가장 최적화된 가중치와 편향 값을 찾아주는 함수


* **경사하강법** \
: 함수의 기울기를 구하고 기울기가 낮은 쪽으로 계속 이동시키면서 최적의 값을 찾아나감


In [27]:
#learning rate(학습률-하이퍼파라미터): 얼마나 학습을 급하게 할 것인가를 결정
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

#최적화를 수행하는 그래프 
train_op = optimizer.minimize(cost)

In [29]:
#파이썬의 with 기능으로 세션 생성 & 초기화 
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    # 최적화 수행하는 tain_op 그래프 실행 & 실행시마다 변화하는 손실값 출력 :학습 100번 수행 
    for step in range(100):
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})
        
        print(step, cost_val, sess.run(W), sess.run(b))
    # 학습에 사용되지 않은 값으로 결과 확인하기 
    print("\n====Test====")
    print("X:2.5, Y:", sess.run(hypothesis, feed_dict={X: 2.5}))
    print("X:5, Y:", sess.run(hypothesis, feed_dict={X: 5}))

0 0.4985831 [0.75730896] [0.6282697]
1 0.05968282 [0.7325127] [0.59969217]
2 0.05188799 [0.7422907] [0.58674866]
3 0.04936405 [0.74811995] [0.57248265]
4 0.0470185 [0.75421494] [0.5587381]
5 0.044785053 [0.7601191] [0.54530454]
6 0.042657737 [0.76588607] [0.532196]
7 0.040631484 [0.771514] [0.5194023]
8 0.03870145 [0.7770067] [0.5069163]
9 0.03686309 [0.7823672] [0.49473035]
10 0.03511207 [0.78759897] [0.48283738]
11 0.033444226 [0.792705] [0.47123033]
12 0.031855617 [0.79768825] [0.45990226]
13 0.03034244 [0.8025516] [0.4488465]
14 0.028901147 [0.8072982] [0.43805653]
15 0.027528316 [0.8119306] [0.42752597]
16 0.026220703 [0.81645167] [0.41724855]
17 0.024975205 [0.820864] [0.40721816]
18 0.023788853 [0.82517034] [0.3974289]
19 0.022658864 [0.8293731] [0.38787496]
20 0.021582572 [0.8334749] [0.3785507]
21 0.020557376 [0.83747804] [0.3694506]
22 0.019580884 [0.841385] [0.36056927]
23 0.018650759 [0.845198] [0.3519014]
24 0.017764838 [0.8489193] [0.34344193]
25 0.016921014 [0.85255116] 